# 1. Configuration

### Imports

In [74]:
import pygad
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
import random

### Read the data

In [3]:
dataset = pd.read_csv("dataset.csv")
dataset

,Equation,Xs,Ys
0,((x ** 4) - 6),"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[-5, 10, 75, 250, 619, 1290, 2395, 4090, 6555,..."
1,(((x / 8) * 2) + 1),"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[1.25, 1.5, 1.75, 2.0, 2.25, 2.5, 2.75, 3.0, 3..."
2,(((x - 1) - 3) / 5),"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[-0.6, -0.4, -0.2, 0.0, 0.2, 0.4, 0.6, 0.8, 1...."
3,(x * 5),"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60..."
4,(x + 2),"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, ..."
...,...,...,...
93,1*x**5 + -2*x**3 + -1*x + -5,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[-7, 9, 181, 887, 2865, 7333, 16109, 31731, 57..."
94,3*x**5 + 2*x**4 + 1*x + -2,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[4, 128, 892, 3586, 10628, 25924, 55228, 10650..."
95,-4*x**5 + -1*x + -5,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[-10, -135, -980, -4105, -12510, -31115, -6724..."
96,4*x + +1,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[5, 9, 13, 17, 21, 25, 29, 33, 37, 41, 45, 49,..."


# 2. Representation

### Tree data structure

#### Node class

In [4]:
class Node:
    def __init__(self, value=None):
        self.value = value
        self.children = []

    def add_child(self, child):
        self.children.append(child)

#### Equation tree class

The `EquationTree` class is a custom implementation of a binary tree data structure that represents a mathematical equation. It is used to parse and evaluate mathematical expressions in a tree-like structure. 

The class has the following methods:

- `build_tree(self, expression)`: Builds the tree structure from a given mathematical expression.
- `print_tree(self)`: Prints the tree structure in a readable format.
- `evaluate(self, x)`: Evaluates the mathematical expression represented by the tree structure for a given value of `x`.


In [259]:
class EquationTree:
    def __init__(self, root=None):
        self.root = root

    def build_tree(self, expression):
        self.root = self._build_tree_recursive(expression)

    def print_tree(self):
        self._print_tree_recursive(self.root)
    
    def evaluate(self, x):
        return self._evaluate_recursive(self.root, x)
     
    def get_array(self):
        return self._get_array_recursive(self.root)

    def replance_random_subtree(self, new_subtree):
        if self.root is None or self.root.children == []:
            self.root = new_subtree.root
        else:
            self._replace_random_subtree_recursive(self.root.children[random.randint(0, 1)], new_subtree.root)
    
    def get_random_subtree(self):
        if self.root is None:
            return None
        return self._get_random_subtree_recursive(self.root)

#recursive functions
        
    def _get_array_recursive(self, root):
        if root is None:
            return []

        array = []

        # If the node has children, surround the value with parentheses
        if root.children:
            array.append("(")

        # Add the left child recursively
        array += self._get_array_recursive(root.children[0]) if root.children else []

        # Add the value of the root node
        array.append(root.value)

        # Add the right child recursively
        array += self._get_array_recursive(root.children[1]) if len(root.children) > 1 else []

        # If the node has children, close the parentheses
        if root.children:
            array.append(")")

        return array

    def _get_random_subtree_recursive(self, current_node):
        if not current_node.children:
            return EquationTree(current_node) 

        if random.choice([True, False]):
            return self._get_random_subtree_recursive(random.choice(current_node.children))
        else:
            return EquationTree(current_node)

    def _replace_random_subtree_recursive(self, current_node, new_subtree_root):
        if not current_node.children:
            current_node.value = new_subtree_root.value
            current_node.children = new_subtree_root.children
            return

        if random.choice([True, False]):
            self._replace_random_subtree_recursive(random.choice(current_node.children), new_subtree_root)
        else:
            current_node.value = new_subtree_root.value
            current_node.children = new_subtree_root.children

    def _evaluate_recursive(self, root, x):
        if root is None:
            return None

        if root.value == 'x':
            return x
        elif root.value.lstrip('-').isdigit():
            return int(root.value)
        elif root.value == '+':
            return self._evaluate_recursive(root.children[0], x) + self._evaluate_recursive(root.children[1], x)
        elif root.value == '-':
            return self._evaluate_recursive(root.children[0], x) - self._evaluate_recursive(root.children[1], x)
        elif root.value == '*':
            return self._evaluate_recursive(root.children[0], x) * self._evaluate_recursive(root.children[1], x)
        elif root.value == '/':
            return self._evaluate_recursive(root.children[0], x) / self._evaluate_recursive(root.children[1], x)
        elif root.value == '**':
            return self._evaluate_recursive(root.children[0], x) ** self._evaluate_recursive(root.children[1], x)
        else:
            return None
       
    def _print_tree_recursive(self, root, depth=0):
        if root is None:
            return

        print('\t' * depth + str(root.value) + "--------")

        for child in root.children:
            self._print_tree_recursive(child, depth + 1)

    def _build_tree_recursive(self, expression):
        
        current_depth = -1  #odvisn v kok ( gres notr

        lowest_depth = 0    #najvecja globina
        lowest_depth_index = -1 #index najvecje globine, oz tm k je element d splitamo drevo

        leaf = False    #ce je list pa v izrazu ni vec operatorjev pol vrne list
        leaf_value = None   #vrednost lista

        priority_lvl = 0   #to je zto d ma ** prednost pred * in /, pa + in - pred tem

        for i in range(len(expression)):
            if expression[i] == '(':    
                current_depth += 1  #povecamo globino
            elif expression[i] == ')':
                current_depth -= 1  #zmanjsamo globino
            elif expression[i] in ['+', '-']:
                #   ce smo v najmajsi globini    ce sploh ni globine v enacbi    prednost pred drugimi operatorji (kle nima tok veze, bl je spodi)
                if current_depth < lowest_depth or lowest_depth_index == -1 or priority_lvl > 1:  
                    lowest_depth = current_depth
                    lowest_depth_index = i
                    priority_lvl = 1 
            elif expression[i] in ['*', '/']:
                if current_depth < lowest_depth or lowest_depth_index == -1 or priority_lvl > 2:
                    lowest_depth = current_depth
                    lowest_depth_index = i
                    priority_lvl = 2
            elif expression[i] in ['**']:
                if current_depth < lowest_depth or lowest_depth_index == -1 or priority_lvl > 3:
                    lowest_depth = current_depth
                    lowest_depth_index = i
                    priority_lvl = 3
            elif expression[i] == 'x' or expression[i].lstrip('-').isdigit():
                leaf_value = expression[i]
                leaf = True

        if lowest_depth_index == -1 and leaf:
            return Node(leaf_value)
        else:
            root = Node(expression[lowest_depth_index])
            # print(expression[:lowest_depth_index])
            # print(expression[lowest_depth_index+1:])
            root.add_child(self._build_tree_recursive(expression[:lowest_depth_index])) #rekurzivno zgradimo levo poddrevo
            root.add_child(self._build_tree_recursive(expression[lowest_depth_index+1:]))   #rekurzivno zgradimo desno poddrevo

            return root
        

Helper function to convert a string equation into an array of `tokens`.

In [6]:
def stringEQtoArray(equation):
    arr = np.array([])
    skip = 0

    for index, char in enumerate(equation):
        if skip != 0:
            skip-=1
            continue
    
        if char == ' ':
            continue
        elif char == 'x' or char.isdigit():
            arr = np.append(arr, char)
        elif char == '*' and equation[index+1] == '*':
            arr = np.append(arr, '**')
            skip = 1
        elif char in ['+', '-'] and equation[index+1].isdigit():
            if char == '+':
                arr = np.append(arr, equation[index+1])
            else:
                arr = np.append(arr, char+equation[index+1])
            skip = 1
        elif char in ['+', '-', '*', '/', '(', ')']:
            arr = np.append(arr, char)

    return arr

Helper function to convert a string sequance array to an array.

In [7]:
def stringArrayToArray(string):
    string = string[1:-1]
    arr = string.split(',')
    arr = [float(i) for i in arr]
    return arr

In [192]:
#vse dela prou :)

for x in range(0,2):
    equation_inputs = dataset.iloc[x].values[0]
    #print(equation_inputs)
    equation = stringEQtoArray(equation_inputs)
    print(equation)
    equation_tree = EquationTree()
    equation_tree.build_tree(equation) 
    equation_tree.print_tree()

    subtree = equation_tree.get_random_subtree()
    print(subtree.get_array())
    subtree.print_tree()

    equation_tree.replance_random_subtree(subtree)
    print(equation_tree.get_array())

    outputs = stringArrayToArray(dataset.iloc[x].values[2])
    sum = 0
    for i in range(100):
        sum += (equation_tree.evaluate(i+1) - outputs[i])

    #print(sum) #more bit 0
    

['(' '(' 'x' '**' '4' ')' '-' '6' ')']
---------
	**--------
		x--------
		4--------
	6--------
['6']
6--------
['(', '(', '6', '**', '4', ')', '-', '6', ')']
['(' '(' '(' 'x' '/' '8' ')' '*' '2' ')' '+' '1' ')']
+--------
	*--------
		/--------
			x--------
			8--------
		2--------
	1--------
['1']
1--------
['(', '(', '(', 'x', '/', '8', ')', '*', '1', ')', '+', '1', ')']


### 3. Genetic algorithm

In [207]:
#trying for the first equation first
true_equation = stringEQtoArray(dataset.iloc[0].values[0])
inputs = stringArrayToArray(dataset.iloc[0].values[1])
outputs = stringArrayToArray(dataset.iloc[0].values[2])

print(true_equation)
print(inputs)
print(outputs)

['(' '(' 'x' '**' '4' ')' '-' '6' ')']
[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65.0, 66.0, 67.0, 68.0, 69.0, 70.0, 71.0, 72.0, 73.0, 74.0, 75.0, 76.0, 77.0, 78.0, 79.0, 80.0, 81.0, 82.0, 83.0, 84.0, 85.0, 86.0, 87.0, 88.0, 89.0, 90.0, 91.0, 92.0, 93.0, 94.0, 95.0, 96.0, 97.0, 98.0, 99.0, 100.0]
[-5.0, 10.0, 75.0, 250.0, 619.0, 1290.0, 2395.0, 4090.0, 6555.0, 9994.0, 14635.0, 20730.0, 28555.0, 38410.0, 50619.0, 65530.0, 83515.0, 104970.0, 130315.0, 159994.0, 194475.0, 234250.0, 279835.0, 331770.0, 390619.0, 456970.0, 531435.0, 614650.0, 707275.0, 809994.0, 923515.0, 1048570.0, 1185915.0, 1336330.0, 1500619.0, 1679610.0, 1874155.0, 2085130.0, 2313435.0, 2559

In [286]:
def model(equation):
    equation_tree = EquationTree()
    equation_tree.build_tree(equation)

    #get the equation length without ()
    equation_length = len(equation) - equation.count('(') - equation.count(')')          

    return np.array([equation_tree.evaluate(i+1) for i in range(100)]), equation_length


In [331]:
def fitness_func(solution):
    
    model_outputs,equation_length = model(solution)
    error = np.sum(np.abs(model_outputs - outputs) ) * np.log(equation_length +1)

    return error

In [231]:
#try the crossover function to only add the parents
def crossover_func(parents, offspring_size, ga_instance):
    
    #build a parent tree
    parent1_tree = EquationTree()
    parent1_tree.build_tree(parents[0])

    parent2_tree = EquationTree()
    parent2_tree.build_tree(parents[1])

    #get a random subtree from parent1
    parent1_subtree = parent1_tree.get_random_subtree()

    #replace a random subtree from parent2 with the subtree from parent1
    parent2_tree.replance_random_subtree(parent1_subtree)

    return parent2_tree.get_array()

In [307]:
#try the mutation function to only change the operators
def mutation_func(offspring, ga_instance):
    
    for i in range(len(offspring)):
        
        if np.random.rand() > 0.5:
            continue

        if offspring[i] in ['+', '-', '*', '/', '**']:
            offspring[i] = np.random.choice(['+', '-', '*', '/', '**'])
        elif offspring[i].isdigit():
            random_number = str(np.random.randint(-10, 10))
            if random_number == '0':
                random_number = '1'
            offspring[i] = random_number

    return offspring

In [325]:
#function to check if the equation is valid
def is_valid(solution):
    #in the equation array search for the following patterns
    # x ** 0, x / 0, number / 0, x - x, x ** x
    #if any of the patterns is found return False

    for i in range(len(solution)):
        if solution[i] == 'x' and i+2 < len(solution):
            if solution[i+1] == '**' and solution[i+2] == '0':
                return False
            elif solution[i+1] == '/' and solution[i+2] == '0':
                return False
            elif solution[i+1] == '-' and solution[i+2] == 'x':
                return False
            elif solution[i+1] == '**' and solution[i+2] == 'x':
                return False
        elif solution[i].isdigit() and i+2 < len(solution):
            if solution[i+1] == '/' and solution[i+2] == '0':
                return False
            
    return True

In [337]:
initial_population = [['x','+', '1']] + [['x','*', '2']] +[['x','/', '2']] + [['x','**', '2']]

In [338]:
for generation in range(0,10000):

    print("Generation = {generation}".format(generation=generation))
    print(initial_population)

    #calculate fitness of each individual
    fitness = []
    for individual in initial_population:
        fitness.append(fitness_func(individual))

    print(fitness)
    #select two parents with the best fitness
    sorted_fitness = np.argsort(fitness)
    parent_indices = sorted_fitness[:2]
    parent1 = initial_population[parent_indices[0]]
    parent2 = initial_population[parent_indices[1]]
    parents = [parent1, parent2]

    #apply crossover
    child = crossover_func(parents, 1, None)
    if is_valid(child):
        initial_population.append(child)

    parents = [parent2, parent1]
    child = crossover_func(parents, 1, None)
    if is_valid(child):
        initial_population.append(child)
    

    #remove the 2 worst individual
    sorted_fitness = np.argsort(fitness)
    initial_population.pop(sorted_fitness[-1])

    
    #apply mutation
    for i in range(len(initial_population)):
        if np.random.rand() < 0.9:
            new_individual = mutation_func(initial_population[i], None)
            if is_valid(new_individual):
                initial_population[i] = new_individual


#plot the best individual
best_index = np.argsort(fitness)[0]
best_individual = initial_population[best_index]
print(best_individual)
best_individual_tree = EquationTree()
best_individual_tree.build_tree(best_individual)
best_individual_tree.print_tree()

x = np.arange(1, 101)
y = [best_individual_tree.evaluate(i) for i in x]

#plot the best individual and the true equation
plt.plot(x, y, label='best individual')
plt.plot(x, outputs, label='true equation')
plt.legend()
plt.show()
    

Generation = 0
[['x', '+', '1'], ['x', '*', '2'], ['x', '/', '2'], ['x', '**', '2']]
[2842357582.010712, 2842350719.853625, 2842361216.8745275, 2841895665.9569983]
Generation = 1
[['x', '+', '1'], ['x', '*', '2'], ['x', '**', '8'], ['(', 'x', '*', 'x', ')'], ['(', '2', '/', '-7', ')']]
[2842357582.010712, 2842350719.853625, 1.6105658890934272e+17, 2841895665.9569983, 2842364754.697737]
Generation = 2
[['x', '+', '1'], ['x', '*', '-10'], ['(', 'x', '*', 'x', ')'], ['(', '2', '/', '-7', ')'], ['(', '(', 'x', '**', 'x', ')', '-', '2', ')'], ['(', 'x', '*', '(', 'x', '*', '2', ')', ')']]
[2842357582.010712, 2842434709.8837876, 2841895665.9569983, 2842364754.697737, 1.7984089707519677e+200, 3672490625.4975147]
Generation = 3
[['x', '**', '1'], ['x', '*', '-10'], ['(', 'x', '/', 'x', ')'], ['(', '2', '+', '-7', ')'], ['(', 'x', '**', '(', 'x', '*', '6', ')', ')'], ['(', 'x', '+', 'x', ')'], ['(', 'x', '**', '(', 'x', '+', '-1', ')', ')']]
[2842357717.86756, 2842434709.8837876, 2842364580.024

ZeroDivisionError: 0.0 cannot be raised to a negative power

In [23]:
#try the ga
ga_instance = pygad.GA(num_generations=100,
                        num_parents_mating=2,
                        fitness_func=fitness_func,
                        sol_per_pop=10,
                        num_genes=100,
                        gene_type=np.array,
                        gene_space=gene_space,
                        crossover_type=crossover_func,
                        mutation_type=mutation_func,
                        initial_population=initial_population)

ga_instance.run()

All values in the sublists inside the 'gene_space' attribute must be numeric of type int/float/None but (x) of type <class 'numpy.str_'> found.
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pygad/pygad.py", line 220, in __init__
    raise TypeError(f"All values in the sublists inside the 'gene_space' attribute must be numeric of type int/float/None but ({val}) of type {type(val)} found.")
TypeError: All values in the sublists inside the 'gene_space' attribute must be numeric of type int/float/None but (x) of type <class 'numpy.str_'> found.


AttributeError: 'tuple' object has no attribute 'tb_frame'